In [1]:
import chipwhisperer as cw
scope = cw.scope()
scope.default_setup()

In [21]:
# change this to match the path of your HEX file
HEX_PATH =  "./binary4.hex"

In [22]:
# program the board
cw.program_target(scope, cw.programmers.STM32FProgrammer, HEX_PATH)

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 17955 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 17955 bytes


In [23]:
## UPDATE THESE KEYS TO MATCH YOUR SETUP
XOR_KEY = [
    0x1f,0xbc,0xcf,0x99,
    0x3d,0xe1,0x90,0x7d,
    0x69,0xae,0xfd,0x73,
    0x02,0x1b,0xed,0x9b
]

DES_KEY = [
    0x38,0xd3,0x8f,0x6d,
    0xd2,0x26,0x84,0x77
]

AES_KEY = [
    0x74, 0xe4, 0x41, 0x66,
    0x26, 0x53, 0x46, 0x77,
    0x47, 0xf3, 0x75, 0x40,
    0x30, 0x6e, 0x16, 0xe0
]


PROTOCOL_NOP = 0x10
PROTOCOL_XOR = 0x20
PROTOCOL_DES = 0x30
PROTOCOL_AES = 0x40

MAGIC_BYTE = 0xFE

RESPONSE_SUCCESS = 0x00
RESPONSE_FAILURE = 0x01
RESPONSE_UNKNOWN_PROTOCOL = 0x02

def response_code_to_string(code):
    code = ord(code)
    if code == RESPONSE_SUCCESS:
        return "SUCCESS"
    elif code == RESPONSE_FAILURE:
        return "FAILURE"
    elif code == RESPONSE_UNKNOWN_PROTOCOL:
        return "UNKNOWN_PROTOCOL"
    else:
        return "UNKNOWN_RESPONSE_CODE"

# reads n bytes from target uart
def read_uart(target, n_bytes):
    response = []
    while len(response) < n_bytes:
        response += target.read()
    return response

In [24]:
target = cw.target(scope, noflush=True) # MUST HAVE noflush=True
print(target.baud) # make sure this baud rate is set in your project

38400


In [25]:
# #convert the message to a list of bytes
from time import sleep
from Crypto.Cipher import DES
from Crypto.Cipher import AES
message = "I am the senate"

message = [ord(c) for c in message]
#put the magic byte at the beginning of the message
message_plaintext = [MAGIC_BYTE] + message

sleep(0.1)

print("===============MESSAGE_NOP=================")

#make a copy of the message to send with the nop protocol
message_nop = message_plaintext.copy()

#no operation on the message

#prepend the protocol byte
message_nop = [PROTOCOL_NOP] + message_nop

#print the hex representation of the message_nop
print("Message_nop :" + " ".join("{:02x}".format(c) for c in message_nop))
#print(bytes(message_nop))
#write the message to the uart
target.write(bytes(message_nop))

#read the 17 byte response
response_nop = read_uart(target, 17)

print("Response_nop:" + " ".join("{:02x}".format(ord(c)) for c in response_nop))

#check the response code
print("Response code: {}".format(response_code_to_string(response_nop[0])))

#convert the response_nop to a printable string (excluding the response code and magic byte)
response_nop = "".join(response_nop[2:])
print("Response_nop: {}".format(response_nop))

sleep(0.1)

print("===============MESSAGE_XOR=================")

#make a copy of the message to send with the xor protocol
message_xor = message_plaintext.copy()

#xor the message with the key
for i in range(len(message_xor)):
    message_xor[i] ^= XOR_KEY[i]

#prepend the protocol byte
message_xor = [PROTOCOL_XOR] + message_xor

#print the hex representation of the message_xor
print("Message_xor :" + " ".join("{:02x}".format(c) for c in message_xor))

# start scoping before sending decrypt request
scope.arm()

#write the message to the uart
target.write(bytes(message_xor))

#read the 17 byte response
response_xor = read_uart(target, 17)

#print the hex representation of the response
print("Response_xor:" + " ".join("{:02x}".format(ord(c)) for c in response_xor))

#check the response code
print("Response code: {}".format(response_code_to_string(response_xor[0])))

#convert the response_xor to a printable string (excluding the response code and magic byte)
response_xor = "".join(response_xor[2:])
print("Response_xor: {}".format(response_xor))

sleep(0.1)

print("===============MESSAGE_DES=================")

enc_des = DES.new(bytearray(DES_KEY), DES.MODE_ECB)

#make a copy of the message to send with the des protocol
message_des = message_plaintext.copy()

#split the message into two 8 byte blocks
message_1 = message_des[:8]
message_2 = message_des[8:]

#encrypt the first block
message_1 = enc_des.encrypt(bytes(message_1))

#encrypt the second block
message_2 = enc_des.encrypt(bytes(message_2))

#prepend the protocol byte
message_des = bytes([PROTOCOL_DES]) + message_1 + message_2

#print the hex representation of the message_des
print("Message_des :" + " ".join("{:02x}".format(c) for c in message_des))

# start scoping before sending decrypt request
scope.arm()

#write the message to the uart
target.write(bytes(message_des))

#read the 17 byte response
response_des = read_uart(target, 17)

#print the hex representation of the response
print("Response_des:" + " ".join("{:02x}".format(ord(c)) for c in response_des))

#check the response code
print("Response code: {}".format(response_code_to_string(response_des[0])))

#convert the response_des to a printable string (excluding the response code and magic byte)
response_des = "".join(response_des[2:])
print("Response_des: {}".format(response_des))

sleep(0.1)

print("===============MESSAGE_AES=================")

enc_aes = AES.new(bytearray(AES_KEY), AES.MODE_ECB)

#make a copy of the message to send with the aes protocol
message_aes = message_plaintext.copy()

#we're using AES-128 and the message is 16 bytes long

#encrypt the message
message_aes = enc_aes.decrypt(bytes(message_aes))

#prepend the protocol byte
message_aes = bytes([PROTOCOL_AES]) + message_aes

#print the hex representation of the message_aes
print("Message_aes :" + " ".join("{:02x}".format(c) for c in message_aes))

# start scoping before sending decrypt request
scope.arm()

#write the message to the uart
target.write(bytes(message_aes))

#read the 17 byte response
response_aes = read_uart(target, 17)

#print the hex representation of the response
print("Response_aes:" + " ".join("{:02x}".format(ord(c)) for c in response_aes))

#check the response code
print("Response code: {}".format(response_code_to_string(response_aes[0])))

#convert the response_aes to a printable string (excluding the response code and magic byte)
response_aes = "".join(response_aes[2:])
print("Response_aes: {}".format(response_aes))

sleep(0.1)

===============MESSAGE_NOP=================
Message_nop :10 fe 49 20 61 6d 20 74 68 65 20 73 65 6e 61 74 65
Response_nop:00 fe 49 20 61 6d 20 74 68 65 20 73 65 6e 61 74 65
Response code: SUCCESS
Response_nop: I am the senate
===============MESSAGE_XOR=================
Message_xor :20 e1 f5 ef f8 50 c1 e4 15 0c 8e 8e 16 6c 7a 99 fe
Response_xor:00 fe 49 20 61 6d 20 74 68 65 20 73 65 6e 61 74 65
Response code: SUCCESS
Response_xor: I am the senate
===============MESSAGE_DES=================
Message_des :30 17 de 6e fe 3b b5 07 36 25 eb be bb e5 a4 c7 be
Response_des:00 fe 49 20 61 6d 20 74 68 65 20 73 65 6e 61 74 65
Response code: SUCCESS
Response_des: I am the senate
===============MESSAGE_AES=================
Message_aes :40 86 1c 46 53 76 47 31 8e 1e 32 bc a4 fd 66 eb 09
Response_aes:00 fe 49 20 61 6d 20 74 68 65 20 73 65 6e 61 74 65
Response code: SUCCESS
Response_aes: I am the senate
